In [1]:
import sys
sys.path.append('..')

In [5]:
from importlib import reload
from os.path import exists

import networkx as nx
import numpy as np
import pandas as pd
import lib.obo
import lib.classification.elastic
import lib.classification.validation
import lib.utils
import data
import pymonad
import json
reload(lib.classification.validation)
reload(lib.classification.elastic)
reload(lib.obo)
reload(lib.utils)
reload(data)

from lib.classification.elastic import annotate_index, collapse_matches, build_synonyms_graph, search_term
from lib.utils import map_series, expand_dataframe, expand, flatten
from lib.classification.validation import correct_ratio, incorrect_ratio, perfect_match
from geo_annotation.import_ontology_es import import_ontology
from elasticsearch import Elasticsearch
from lib.obo import read_ontology, Ontology
pd.set_option('display.width', 512)

In [3]:
import seaborn
%matplotlib inline

In [21]:
es = Elasticsearch()

In [13]:
with open('../data/geo-annotation/samples-affumetrix-preprocessed.json') as f:
    samples_ids = [int(s[3:]) for s in json.load(f) if 'Accession' not in s]

In [22]:
normal_samples = search_term(
    es=es,
    term=['normal', 'control', 'reference', 'healthy'],
    index='samples',
    ids=samples_ids
)

In [24]:
len(normal_samples), len(samples_ids)

(38948, 203770)

In [25]:
normal_samples[:10]

[1434757,
 629725,
 629735,
 629724,
 629729,
 629722,
 629727,
 349849,
 349848,
 1434760]

In [28]:
normal_samples_gsm = ['GSM{}'.format(s) for s in normal_samples] 
normal_samples_gsm[:10]

['GSM1434757',
 'GSM629725',
 'GSM629735',
 'GSM629724',
 'GSM629729',
 'GSM629722',
 'GSM629727',
 'GSM349849',
 'GSM349848',
 'GSM1434760']

In [34]:
with open('../data/geo-annotation/normal_samples.txt', 'w') as f:
    for s in normal_samples_gsm:
        f.write('{}\n'.format(s))

In [35]:
normal_samples_all =  search_term(
    es=es,
    term=['normal', 'control', 'reference', 'healthy'],
    index='samples'
)

In [36]:
with open('../data/geo-annotation/normal_samples_all.txt', 'w') as f:
    for s in normal_samples_all:
        f.write('GSM{}\n'.format(s))